<a href="https://colab.research.google.com/github/Miliyas/Generative_AI/blob/main/Rag_with_llama_index_and_mistral_mib.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%pip install llama-index-llms-huggingface
%pip install llama-index

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 49.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 136.1/136.1 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.3/268.3 kB 36.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 70.4 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.wh

In [2]:
!pip install weaviate-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 306.8/306.8 kB 6.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 223.7/223.7 kB 12.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 19.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.8/302.8 kB 20.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 294.6/294.6 kB 22.4 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.25.3 which is incompatible.


In [1]:
WEAVIATE_CLUSTER="https://mistral-z66pwykr.weaviate.network"
WEAVIATE_API_KEY="X8Uf9AjZtnKrgqtWrVAZyzNVspYOJu7zrUhr"

In [2]:
import weaviate

In [3]:
client=weaviate.Client(url=WEAVIATE_CLUSTER,auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY))

In [4]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader


In [7]:
!mkdir Data

In [5]:
loader=SimpleDirectoryReader('/content/Data')

In [6]:
documents=loader.load_data()

In [7]:
documents

[Document(id_='35778182-9345-4c31-abc2-7d7d35105838', embedding=None, metadata={'page_label': '1', 'file_name': 'MachineTranslationwithAttention (1).pdf', 'file_path': '/content/Data/MachineTranslationwithAttention (1).pdf', 'file_type': 'application/pdf', 'file_size': 482137, 'creation_date': '2024-04-15', 'last_modified_date': '2024-04-15'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, text='See discussions, st ats, and author pr ofiles f or this public ation at : https://www .researchgate.ne t/public ation/355917108\nNeu ral Machine T ranslation with Attention\nTechnic al R eport  · August 2021\nDOI: 10.13140/RG.2.2.29381.37607/1\nCITATIONS\n0READS\n1,667\n2 author s:\nMohammad W asil Saleem\nUniv ersität P otsdam\n3 PUBLICA TIONS \xa0\xa0\xa00 CITATIONS

In [11]:
%pip install llama-index-embeddings-huggingface
%pip install llama-index-embeddings-instructor

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 163.3/163.3 kB 4.4 MB/s eta 0:00:00


In [8]:
from llama_index.embeddings.huggingface import HuggingFaceEmbedding
embed_model=HuggingFaceEmbedding(model_name="sentence-transformers/all-mpnet-base-v2")


/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [9]:
from llama_index.core import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

In [10]:
system_prompt=""""<|SYSTEM|># You are a Q&A assistant. Your goal is to answer questions as
accurately as possible based on the instructions and context provided

"""

In [11]:
# This will wrap the default prompts that are internal to llama-index
query_wrapper_prompt = PromptTemplate("<|USER|>{query_str}<|ASSISTANT|>")

In [12]:
query_wrapper_prompt

PromptTemplate(metadata={'prompt_type': <PromptType.CUSTOM: 'custom'>}, template_vars=['query_str'], kwargs={}, output_parser=None, template_var_mappings=None, function_mappings=None, template='<|USER|>{query_str}<|ASSISTANT|>')

In [13]:
import torch
llm=HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.9, "do_sample": False},
    system_prompt=system_prompt,
    query_wrapper_prompt=query_wrapper_prompt,
    tokenizer_name="mistralai/Mistral-7B-Instruct-v0.1",
    model_name="mistralai/Mistral-7B-Instruct-v0.1",
    device_map="auto",
    stopping_ids=[50278, 50279, 50277, 1, 0],
    tokenizer_kwargs={"max_length": 4096},
    # uncomment this if using CUDA to reduce memory usage
    model_kwargs={"torch_dtype": torch.float16}

)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [14]:
from llama_index.core import ServiceContext

Service_Context=ServiceContext.from_defaults(
    chunk_size=1024,
    llm=llm,
    embed_model=embed_model
)

<ipython-input-14-f0b118d025f9>:3: DeprecationWarning: Call to deprecated class method from_defaults. (ServiceContext is deprecated, please use `llama_index.settings.Settings` instead.) -- Deprecated since version 0.10.0.
  Service_Context=ServiceContext.from_defaults(


In [15]:
from llama_index.core import VectorStoreIndex

In [16]:
index=VectorStoreIndex.from_documents(documents,service_context=Service_Context)

In [17]:
query_engine=index.as_query_engine(k=1)

In [18]:
result=query_engine.query("which matrics we can use for evaluating the machine translation task?")

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:410: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [19]:
result.response

'We can use BLEU (Bilingual Evaluation Understudy) score for evaluating the machine translation task. BLEU score is a widely used metric for evaluating the quality of machine translation. It measures the similarity between the generated translation and a set of human reference translations. The higher the BLEU score, the better the quality of the translation. In addition to BLEU score, we can also use other metrics such as perplexity, ROUGE (Recall-Oriented Understudy for Gisting Evaluation), and METEOR (Metric for Evaluation of Translation with Explicit ORdering).'

In [20]:
result2=query_engine.query("what is attention mechnisam?")
print(result2)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


Attention is a mechanism used in machine translation models to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which helps to improve the translation accuracy. The attention mechanism works by creating an alignment between each time step of the decoder output and all of the encoder hidden state. This alignment is learned during the training process, and each output of the decoder can selectively pick out specific elements from the sequence to produce the output. The attention mechanism helps to address the bottle-neck problem in RNN Encoder-Decoder models, where the complete sequence of information of the source sentence must be captured by one single vector, which is difficult for the decoder to summarize large input sequences at once.


In [21]:
from IPython.display import Markdown, display

In [22]:
display(Markdown(f"<b>{result}</b>"))

<b>We can use BLEU (Bilingual Evaluation Understudy) score for evaluating the machine translation task. BLEU score is a widely used metric for evaluating the quality of machine translation. It measures the similarity between the generated translation and a set of human reference translations. The higher the BLEU score, the better the quality of the translation. In addition to BLEU score, we can also use other metrics such as perplexity, ROUGE (Recall-Oriented Understudy for Gisting Evaluation), and METEOR (Metric for Evaluation of Translation with Explicit ORdering).</b>

In [23]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [24]:
!pip install llama-index-vector-stores-weaviate

In [25]:
from llama_index.vector_stores.weaviate import WeaviateVectorStore
from llama_index.core import StorageContext



In [26]:
vector_store=WeaviateVectorStore(weaviate_client=client,index_name="Llamaindex")

In [27]:
storage_context = StorageContext.from_defaults(vector_store=vector_store)


In [28]:
index = VectorStoreIndex.from_documents(
    documents, storage_context=storage_context,embed_model=embed_model
)

In [29]:
query_engine = index.as_query_engine(llm=llm)

In [30]:
result3=query_engine.query("what is attention?")

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [32]:
result3.response

'Attention is a mechanism used in machine translation models to retain and utilize all the hidden state of the input sequence during the decoding phase. It allows the model to focus and pay more attention to the relevant part of the input sequence, which improves the translation accuracy. The attention mechanism works by creating an alignment between each time step of the decoder output and all of the encoder hidden state. The alignment score is calculated by multiplying the decoder hidden state at time step t with all of the encoder hidden state, which gives us the alignment score of each of the encoder output with respect to the decoder input and hidden state at that time step. The attention score quantifies the amount of attention the decoder will place on each of the encoder outputs when producing the next output, so instead of looking at the entire sequence, it just concentrates on few relevant parts of the sequence when predicting the next word. After calculating the alignment sc